# Web Scraping: Selenium

A menudo, los datos están disponibles públicamente para nosotros, pero no en una forma que sea fácilmente utilizable. Ahí es donde entra en juego el web scraping, podemos usar web scraping para obtener nuestros datos deseados en un formato conveniente que luego se puede usar. A continuación, mostraré cómo se puede extraer información de interés de un sitio web usando el paquete Selenium en Python. Selenium nos permite manejar una ventana del navegador e interactuar con el sitio web mediante programación. 

Selenium también tiene varios métodos que facilitan la extracción de datos.
En este Jupyter Notebook vamos a usar Python 3 en Windows.

En primer lugar, tendremos que descargar un controlador.

Usaremos ChromeDriver para Google Chrome. Para obtener una lista completa de controladores y plataformas compatibles, consulte [Selenium](https://www.selenium.dev/downloads/). Si desea utilizar Google Chrome, diríjase a [chrome](https://chromedriver.chromium.org/) y descargue el controlador que corresponde a su versión actual de Google Chrome.

Como saber cual es la version de chrome que utilizo simple utilizamos pegamos el siguiente enlace en la barra de chrome chrome://settings/help

Antes de comenzar se preguntaran si ya se BeautifulSoup cual es la diferencia con Selenium.

A diferencia BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un Document Object Model (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM



## Comencemos importando las bibliotecas que usaremos:

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import re # Expresiones regulares 
import time
import pandas as pd

El objeto driver es con el que trabajaremos a partir de ahora

In [3]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = 'chromedriver.exe'
options  = webdriver.ChromeOptions()

In [4]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

In [5]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://insolvencyinsider.ca/filing/'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

Ahora si queremos hacer click en el boton de "Load more".

Selenium proporciona varios métodos para localizar elementos en la página web. Usaremos el método find_element_by_xpath() para crear un objeto de botón, con el que luego podremos interactuar:

In [5]:
# loadMore = driver.find_element_by_xpath(xpath="/html/body/div[2]/div/main/div/div/div/button")

In [6]:
loadMore = driver.find_element(By.XPATH, '/html/body/div[2]/div/main/div/div/div/button')

Antes de continuar, necesitaremos saber cuántas páginas hay para saber cuántas veces debemos hacer clic en el botón. Necesitaremos una forma de extraer el código fuente del sitio web. Afortunadamente, este proceso es relativamente sencillo con las bibliotecas urllib3 y re.

In [7]:
url = "https://insolvencyinsider.ca/filing/"
http = urllib3.PoolManager()
r = http.request("GET", url)
text = str(r.data)


```text``` ahora es una cadena. Ahora, necesitamos una forma de extraer total_pages de nuestra cadena de texto. Imprima texto para ver cómo podemos extraerlo usando RegEx con el paquete re. Podemos totalizar_páginas así:

In [8]:
totalPagesObj = re.search(pattern='"total_pages":\d+', string=text)
totalPagesStr = totalPagesObj.group(0)
totalPages = int((re.search(pattern="\d+", string=totalPagesStr)).group(0))

El método de búsqueda toma un patrón y una cadena. En este caso nuestro patrón es '"total_pages":\d+' . Si no está familiarizado con RegEx, todo esto significa que estamos buscando la cadena "total_pages": con dos o más dígitos después de los dos puntos. \d se refiere a un dígito entre 0 y 9, mientras que + indica que Python debe buscar una o más de las expresiones regulares anteriores. Puedes leer más sobre el paquete re aquí. El método search() devuelve un objeto Match. re proporciona el método group() que devuelve uno o más subgrupos de la coincidencia. Pasamos 0 como argumento para indicar que queremos el parche completo. La tercera línea simplemente extrae el entero correspondiente a total_pages de la cadena.

In [9]:
print(totalPagesObj)
print(totalPagesStr)
print(totalPages)

<re.Match object; span=(116405, 116422), match='"total_pages":106'>
"total_pages":106
106


Con eso completo, ahora podemos cargar todas las páginas de Insolvency Insider. Podemos hacer clic en el botón Cargar más accediendo al método click() del objeto. Esperamos tres segundos entre clics para no sobrecargar el sitio web.

# Dale click a load more en función al número de páginas totales

Una vez que ejecute esto, debería ver que se hace clic en el botón Cargar más y que se cargan las páginas restantes.
Una vez que se carga cada página, podemos comenzar a raspar el contenido. Ahora, eliminar ciertos elementos como el nombre de presentación, la fecha y la hiperreferencia es bastante sencillo. Podemos usar los métodos find_elements_by_class_name() y find_elements_by_xpath() de Selenium (importante la ```s``` extra después de element):

In [11]:
# filingNamesElements = driver.find_elements_by_class_name("filing-name")

# filingDateElements = driver.find_elements_by_class_name("filing-date")

# filingHrefElements = driver.find_elements_by_xpath("//*[@id='content']/div[2]/div/div[1]/h3/a")

In [13]:
filingNamesElements = driver.find_elements(By.CLASS_NAME, "filing-name")
filingDateElements = driver.find_elements(By.CLASS_NAME, "filing-date")
filingHrefElements = driver.find_elements(By.XPATH, "//*[@id='content']/div[2]/div/div[1]/h3/a")

También nos gustaría conocer los metadatos de presentación, es decir, el tipo de archivo, el sector de la empresa y el lugar en la que operan. Extraer estos datos requiere un poco más de trabajo.

In [14]:
filingMetas = []
for i in tqdm(range(len(filingNamesElements) + 1)):
    filingMetai = driver.find_elements(By.XPATH, ("//*[@id='content']/div[2]/div[%d]/div[2]/div[1]" %(i)))
    for element in filingMetai:
        filingMetaTexti = element.text
        filingMetas.append(filingMetaTexti)

100%|██████████| 963/963 [00:39<00:00, 24.18it/s]


De cada elemento de la presentación de Metas podemos extraer el tipo de presentación, la industria y la provincia, así:

In [45]:
metaDict = {"Filing Type": [], "Industry": [], "Province": []}
for filing in tqdm(filingMetas):
    filingSplit = filing.split("\n")
  
    for item in tqdm(filingSplit):
        itemSplit = item.split(":")

        
        if itemSplit[0] == "Filing Type":
            metaDict["Filing Type"].append(itemSplit[1])
        if itemSplit[0] == "Industry":
            metaDict["Industry"].append(itemSplit[1])
        if itemSplit[0] == "Province":
            metaDict["Province"].append(itemSplit[1])
            
    if "Filing Type" not in filing:
        metaDict["Filing Type"].append("NA")
    if "Industry" not in filing:
        metaDict["Industry"].append("NA")
    if "Province" not in filing:
        metaDict["Province"].append("NA")

Ahora, todavía tenemos que poner nuestros nombres y fechas de presentación en las listas. Hacemos esto agregando el texto de cada elemento a una lista usando el método text() de antes:

In [15]:
filingName = []
filingDate = []
filingLink = []
# para cada elemento en la lista de elementos de nombre de archivo, agrega el
# texto del elemento a la lista de nombres de archivo.
for element in tqdm(filingNamesElements):
    filingName.append(element.text)
# para cada elemento en la lista de elementos de la fecha de presentación, agrega el
# texto del elemento a la lista de fechas de presentación.
for element in tqdm(filingDateElements):
    filingDate.append(element.text)
for link in tqdm(filingHrefElements):
    if link.get_attribute("href"):
        filingLink.append(link.get_attribute("href"))

100%|██████████| 962/962 [00:11<00:00, 84.39it/s]


Una vez que tengamos eso, estamos listos para poner todo en un diccionario y luego crear un DataFrame de pandas:

In [18]:
# Crea un diccionario final con nombres y fechas de archivo.
# fullDict = {

#     "Filing Name": filingName,
#     "Filing Date": filingDate, 
#     "Filing Type": metaDict["Filing Type"],
#     "Industry": metaDict["Industry"],
#     "Province": metaDict["Province"],
#     "Link": filingLink
# }
fullDict = {
    "Filing Name": filingName,
    "Filing Date": filingDate, 
    "Link": filingLink
}
# Crea un DataFrame.
df = pd.DataFrame(fullDict)
df["Filing Date"] = pd.to_datetime(df["Filing Date"], infer_datetime_format=True)

In [19]:
df

,Filing Name,Filing Date,Link
0,2737524 Ontario Inc. o/a Estrada Automotive,2022-07-05,https://insolvencyinsider.ca/filing/2737524-on...
1,Premium Comfort Heating & Air Conditioning Ltd,2022-06-28,https://insolvencyinsider.ca/filing/premium-co...
2,"Sproutly Inc. and its wholly-owned subsidiary,...",2022-06-24,https://insolvencyinsider.ca/filing/sproutly-i...
3,Revlon Canada Inc. and Elizabeth Arden (Canada...,2022-06-20,https://insolvencyinsider.ca/filing/revlon-can...
4,Zenabis Global Inc. & al. (the “Zenabis Group”),2022-06-17,https://insolvencyinsider.ca/filing/zenabis-gl...
...,...,...,...
957,ReidBuilt,2017-11-02,https://insolvencyinsider.ca/filing/reidbuilt/
958,Spareparts,2017-10-31,https://insolvencyinsider.ca/filing/spareparts/
959,BuildDirect,2017-10-31,https://insolvencyinsider.ca/filing/builddirect/
960,1735549 Ontario,2017-10-27,https://insolvencyinsider.ca/filing/1735549-on...


------------------------

# Ahora algo más visual

In [11]:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

In [12]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://www.filmaffinity.com/es/main.html'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

La página de Filmaffinity se ha abierto

Pero....

Nos hemos encontrado con un pop-up que nos pide aceptar cookies

1. Buscamos el botón
2. Hacemos click en el botón

Vamos a quitar el boton para seguir

In [13]:
elements_by_tag = driver.find_elements(By.TAG_NAME,'button')
elements_by_class_name = driver.find_elements(By.CLASS_NAME, 'css-2tkghh')
element_by_xpath = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div[2]/div/button[2]')

Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [14]:
dir(element_by_xpath)
# obtenemos todos sus métodos y atributos:

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 '_w3c',
 'clear',
 'click',
 'find_element',
 'find_element_by_class_name',
 'find_element_by_css_selector',
 'find_element_by_id',
 'find_element_by_link_text',
 'find_element_by_name',
 'find_element_by_partial_link_text',
 'find_element_by_tag_name',
 'find_element_by_xpath',
 'find_elements',
 'find_elements_by_class_name',
 'find_elements_by_css_selector',
 'find_elements_by_id',
 'find_elements_by_link_text',
 'find_elements_by_name',
 'find_elements_by_partial_link_text',
 'find_elements_by_tag_name',
 'find_elements_by_xpath',
 'get_attribute',
 'get_property',
 'id',
 

Podemos evaluar que tipo de elemento es (tag)

In [27]:
element_by_xpath.tag_name

'button'

Podemos sacar el valor que tiene (el texto)

In [28]:
element_by_xpath.text

'ACEPTO'

In [15]:
from tqdm import tqdm
for i in tqdm(range(0,len(elements_by_tag))):
    print(elements_by_tag[i].text)

100%|██████████| 3/3 [00:00<00:00, 55.78it/s]

socios
MÁS OPCIONES
ACEPTO


Incluso podemos guardar una imagen del elemento

In [30]:
type(element_by_xpath)
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos

selenium.webdriver.remote.webelement.WebElement

In [31]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath
element_by_xpath.screenshot('mi_imagen.png')

True

Evaluamos que elementos hemos encontrado por el tag:

In [ ]:
for index, element in enumerate(elements_by_tag):
    print('Elemento:', index)
    print('Texto del elemento',index, 'es', element.text)
    print('El tag del elemento',index, 'es', element.tag_name)
    element.screenshot('mi_imagen'+str(index)+'.png')

Basta de tonterias seguimos

Instanciamos el elemento del tag [2] en la variable boton aceptar

In [33]:
boton_aceptar = elements_by_tag[2]

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

In [34]:
boton_aceptar.click()

Buscamos una película por título

In [35]:
from selenium.webdriver.common.keys import Keys

In [38]:
buscador = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div[2]/form/div/input')

In [39]:
buscador.send_keys('al filo del mañana')

In [40]:
# una vez escrita la búsqueda deberíamos poder activarla:
buscador.send_keys(Keys.ENTER)

In [41]:
# volvemos a la página anterior
driver.back()

In [77]:
# buscador.clear()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona lateral

In [78]:
menu_lateral = driver.find_element(By.ID, 'lsmenu') ####### si se corrompe el navegador seleciona cualquier pagina distinta a proximos extrenos y ejecuta desde aqui
menu_lateral

WebDriverException: Message: chrome not reachable
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x003C6463+2188387]
	Ordinal0 [0x0035E461+1762401]
	Ordinal0 [0x00273C40+801856]
	Ordinal0 [0x00268932+756018]
	Ordinal0 [0x00269168+758120]
	Ordinal0 [0x0026AA22+764450]
	Ordinal0 [0x00264379+738169]
	Ordinal0 [0x002750D0+807120]
	Ordinal0 [0x002CC402+1164290]
	Ordinal0 [0x002BC5F6+1099254]
	Ordinal0 [0x00296BE0+945120]
	Ordinal0 [0x00297AD6+948950]
	GetHandleVerifier [0x006671F2+2712546]
	GetHandleVerifier [0x0065886D+2652765]
	GetHandleVerifier [0x0045002A+520730]
	GetHandleVerifier [0x0044EE06+516086]
	Ordinal0 [0x0036468B+1787531]
	Ordinal0 [0x00368E88+1805960]
	Ordinal0 [0x00368F75+1806197]
	Ordinal0 [0x00371DF1+1842673]
	BaseThreadInitThunk [0x76806739+25]
	RtlGetFullPathName_UEx [0x77AD8FEF+1215]
	RtlGetFullPathName_UEx [0x77AD8FBD+1165]


In [47]:
mis_secciones = menu_lateral.find_elements(By.TAG_NAME, 'a')

2. Vemos con cuál nos tenemos que quedar

In [48]:
for a in tqdm(mis_secciones):
    if a.text == 'Próximos estrenos':
        a.click()
        break

 11%|█         | 5/45 [00:01<00:09,  4.17it/s]


Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [49]:
cajon_central = driver.find_elements(By.ID, 'main-wrapper-rdcat')

In [50]:
type(cajon_central)

list

In [51]:
for semana in cajon_central:
    print(semana.find_element(By.TAG_NAME, 'div').text)
    print(semana.find_element(By.TAG_NAME, 'div').get_attribute('id'))

14 de julio de 2022
2022-07-14
15 de julio de 2022
2022-07-15
22 de julio de 2022
2022-07-22
29 de julio de 2022
2022-07-29
5 de agosto de 2022
2022-08-05
12 de agosto de 2022
2022-08-12
19 de agosto de 2022
2022-08-19
26 de agosto de 2022
2022-08-26


El siguiente comando es para buscar un boton y darle click en un solo paso

In [54]:
driver.find_element(By.XPATH, '//*[@id="mt-content-cell"]/div[5]/a[2]/div').click()

In [56]:
cajon_central = driver.find_elements(By.ID, 'main-wrapper-rdcat')
for semana in cajon_central:
    print(semana.find_element(By.TAG_NAME, 'div').text)
    print(semana.find_element(By.TAG_NAME, 'div').get_attribute('id'))

8 de julio de 2022
2022-07-08
1 de julio de 2022
2022-07-01
24 de junio de 2022
2022-06-24
17 de junio de 2022
2022-06-17
9 de junio de 2022
2022-06-09
3 de junio de 2022
2022-06-03


In [57]:
for semana in cajon_central:
    fecha = semana.find_element(By.TAG_NAME, 'div').get_attribute('id')
    if fecha == '2022-06-17':
        print('ok')
        break

ok


Buscamos cómo acceder a las películas

In [58]:
caratulas = semana.find_elements(By.CLASS_NAME, 'mc-poster')
lista_pelis = []
for peli in caratulas:
    lista_pelis.append(peli.find_element(By.TAG_NAME, 'a').get_attribute('href'))

In [59]:
lista_pelis

['https://www.filmaffinity.com/es/film463196.html',
 'https://www.filmaffinity.com/es/film695730.html',
 'https://www.filmaffinity.com/es/film696765.html',
 'https://www.filmaffinity.com/es/film300039.html',
 'https://www.filmaffinity.com/es/film808865.html',
 'https://www.filmaffinity.com/es/film754398.html',
 'https://www.filmaffinity.com/es/film428934.html',
 'https://www.filmaffinity.com/es/film316289.html']

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [60]:
# Accedemos a la página de la primera pelicula
driver.get(lista_pelis[0])

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [61]:
# titulo, nota, numero de votos y ficha técnica
titulo = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
nota = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
votos = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
ficha = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

In [62]:
titulo

'Lightyear'

2. Creamos una lista a partir de la ficha técnica

In [63]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'
ficha_names = []
ficha_values = []

for name in ficha.find_elements(By.TAG_NAME, 'dt'):
    ficha_names.append(name.text)
for value in ficha.find_elements(By.TAG_NAME, 'dd'):
    ficha_values.append(value.text)

In [64]:
ficha_values

['Lightyear',
 '2022',
 '100 min.',
 ' Estados Unidos',
 'Angus MacLane',
 'Jason Headley, Angus MacLane, Matthew Aldrich. Personaje: John Lasseter, Pete Docter, Andrew Stanton, Joe Ranft',
 'Michael Giacchino',
 'Animación, Jeremy Lasky, Ian Megibben',
 'Animación',
 'Pixar Animation Studios, Walt Disney Pictures. Distribuidora: Walt Disney Pictures',
 'Animación. Aventuras. Ciencia ficción. Comedia | Cine familiar. Aventura espacial. Robots. Pixar. Spin-off',
 'Toy Story | Pixar (Películas)',
 'La historia del origen de Buzz Lightyear, el héroe que inspiró el juguete, y que nos da a conocer al legendario Guardián Espacial que acabaría contando con generaciones de fans. (FILMAFFINITY)']

3. Creamos un dataframe con la info

In [65]:
columns = ['Titulo', 'Nota', 'Votos']
columns.extend(ficha_names)
len(columns)

16

In [66]:
values = [titulo, nota, votos]
values.extend(ficha_values)
len(values)

16

In [67]:
pd.DataFrame([values],columns=columns)

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Productora,Género,Grupos,Sinopsis
0,Lightyear,"6,0",2.329,Lightyear,2022,100 min.,Estados Unidos,Angus MacLane,"Jason Headley, Angus MacLane, Matthew Aldrich....",Michael Giacchino,"Animación, Jeremy Lasky, Ian Megibben",Animación,"Pixar Animation Studios, Walt Disney Pictures....",Animación. Aventuras. Ciencia ficción. Comedia...,Toy Story | Pixar (Películas),"La historia del origen de Buzz Lightyear, el h..."


Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

In [68]:
def sacar_info(driver):
    
    titulo = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
    try:
        nota = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
        votos = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
    except:
        nota = None
        votos = None
    ficha = driver.find_element(By.XPATH, '/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')
    
    return titulo, nota, votos, ficha

def sacar_ficha(ficha):
    
    ficha_names = []
    ficha_values = []

    for name in ficha.find_elements(By.TAG_NAME, 'dt'):
        ficha_names.append(name.text)
    for value in ficha.find_elements(By.TAG_NAME, 'dd'):
        ficha_values.append(value.text)
        
    return ficha_names, ficha_values

def montar_df(ficha_names, ficha_values, titulo, nota, votos):
    
    columns = ['Titulo', 'Nota', 'Votos']
    columns.extend(ficha_names)
    values = [titulo, nota, votos]
    values.extend(ficha_values)
    
    return pd.DataFrame([values], columns = columns)
    
def nueva_pelicula(driver):
    
    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_names, ficha_values = sacar_ficha(ficha)
    df_peli = montar_df(ficha_names, ficha_values, titulo, nota, votos)
    
    return df_peli


Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

In [69]:
driver.execute_script('window.open("");')

Movernos a otra ventana

In [70]:
driver.switch_to.window(driver.window_handles[0])

Cerrar ventana

In [71]:
driver.close()

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

In [72]:
driver.switch_to.window(driver.window_handles[-1])

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [73]:
# para abrir todos los links en lista_pelis
for link in lista_pelis:
    driver.execute_script('window.open("'+link+'");')
    driver.get(link)

In [75]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:
df_peliculas = pd.DataFrame()

for link in lista_pelis:
    driver.execute_script('window.open("");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    nueva_peli = nueva_pelicula(driver)
    df_peliculas = df_peliculas.append(nueva_peli)

C:\Users\Alex\AppData\Local\Temp\ipykernel_3496\2147100405.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_peliculas = df_peliculas.append(nueva_peli)
C:\Users\Alex\AppData\Local\Temp\ipykernel_3496\2147100405.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_peliculas = df_peliculas.append(nueva_peli)
C:\Users\Alex\AppData\Local\Temp\ipykernel_3496\2147100405.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_peliculas = df_peliculas.append(nueva_peli)
C:\Users\Alex\AppData\Local\Temp\ipykernel_3496\2147100405.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_peliculas = df_peliculas.append(nueva_peli)
C:\Users\Ale

In [76]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 0
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Titulo           8 non-null      object
 1   Nota             8 non-null      object
 2   Votos            8 non-null      object
 3   Título original  8 non-null      object
 4   Año              8 non-null      object
 5   Duración         8 non-null      object
 6   País             8 non-null      object
 7   Dirección        8 non-null      object
 8   Guion            8 non-null      object
 9   Música           7 non-null      object
 10  Fotografía       8 non-null      object
 11  Reparto          8 non-null      object
 12  Productora       8 non-null      object
 13  Género           8 non-null      object
 14  Grupos           1 non-null      object
 15  Sinopsis         8 non-null      object
dtypes: object(16)
memory usage: 1.1+ KB


In [347]:
df_peliculas

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Productora,Género,Grupos,Sinopsis
0,Lightyear,"6,0",2.269,Lightyear,2022,100 min.,Estados Unidos,Angus MacLane,"Jason Headley, Angus MacLane, Matthew Aldrich....",Michael Giacchino,"Animación, Jeremy Lasky, Ian Megibben",Animación,"Pixar Animation Studios, Walt Disney Pictures....",Animación. Aventuras. Ciencia ficción. Comedia...,Toy Story | Pixar (Películas),"La historia del origen de Buzz Lightyear, el h..."
0,El insoportable peso de un talento descomunal,"6,2",1.302,The Unbearable Weight of Massive Talent,2022,105 min.,Estados Unidos,Tom Gormican,"Kevin Etten, Tom Gormican",Mark Isham,Nigel Bluck,"Nicolas Cage, Pedro Pascal, Tiffany Haddish, S...","Lionsgate, Saturn Films. Distribuidora: Lionsgate",Acción. Comedia | Comedia negra. Secuestros / ...,NaN,Nicolas Cage se interpreta a sí mismo en esta ...
0,La brigada de la cocina,"5,8",75,La brigade,2022,97 min.,Francia,Louis-Julien Petit,"Liza Benguigui, Sophie Bensadoun, Louis-Julien...",Laurent Perez del Mar,David Chambille,"Audrey Lamy, François Cluzet, Chantal Neuwirth...","Odyssée Pictures, Apollo Films, France 3 Ciném...",Comedia | Cocina,NaN,Cathy es una estricta chef de 40 años. Cuando ...
0,Estamos hechos para entendernos,"6,4",32,On est fait pour s'entendre,2021,93 min.,Francia,Pascal Elbé,Pascal Elbé,Christophe Minck,Rémy Chevrin,"Sandrine Kiberlain, Pascal Elbé, Valérie Donze...","Jerico, Pere Films, France 3 Cinéma, Canal+, C...",Romance. Comedia | Comedia romántica. Discapac...,NaN,"Antoine, un profesor de historia de cincuenta ..."
0,Nosotros no nos mataremos con pistolas,"5,5",109,Nosotros no nos mataremos con pistolas,2022,88 min.,España,María Ripoll,"Antonio Escámez, Victor Sanchez Rodriguez. Obr...",Simon Smith,Joan Bordera,"Ingrid García Jonsson, Elena Martin, Joe Manjó...","Turanga Films, À Punt Media, Un Capricho de Pr...",Drama. Comedia | Amistad,NaN,Mientras el pueblo se prepara para celebrar su...
0,Tenéis que venir a verla,"6,6",383,Tenéis que venir a verla,2022,64 min.,España,Jonás Trueba,Jonás Trueba,NaN,Santiago Racaj,"Itsaso Arana, Francesco Carril, Irene Escolar,...",Los Ilusos Films,Drama. Comedia | Comedia dramática,NaN,Dos parejas de amigos se reencuentran. Escucha...
0,Rendir los machos,"6,0",79,Rendir los machos,2021,80 min.,España,David Pantaleón,"Amos Milbor, David Pantaleón",Pedro Perles,Cris Noda,"Alejandro Benito, Lili Quintana, José Mentado,...",Coproducción España-Francia; Noodles Productio...,Comedia. Drama | Familia,NaN,Alejandro y Julio son miembros de los hermanos...
0,Comuneros,"8,1",38,Comuneros,2021,84 min.,España,Pablo García Sanz,"Pablo García Sanz, Carlos Belloso, Nerea Mugüerza","Héctor Castrillejo, Carlos Herrero",Víctor Hugo Martín Caballero,Documental,"Visual Creative, Plan Secreto S.L",Documental | Documental sobre Historia. Siglo XVI,NaN,Largometraje documental que narra los hechos a...


Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes